In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("laptop_price.csv", encoding="latin-1")

In [ ]:
df

In [ ]:
df = df.drop("Product", axis=1)

In [ ]:
df = df.join(pd.get_dummies(df.Company))
df = df.drop("Company", axis=1)

In [ ]:
df = df.join(pd.get_dummies(df.TypeName))
df = df.drop("TypeName", axis=1)

In [ ]:
df

In [ ]:
df["ScreenResolution"] = df.ScreenResolution.str.split(" ").apply(lambda x: x[-1])
df["Screen Width"] = df.ScreenResolution.str.split("x").apply(lambda x: x[0])
df["Screen Height"] = df.ScreenResolution.str.split("x").apply(lambda x: x[1])

In [ ]:
df = df.drop("ScreenResolution", axis=1)

In [ ]:
df["CPU Brand"] = df.Cpu.str.split(" ").apply(lambda x: x[0])
df["CPU Frequency"] = df.Cpu.str.split(" ").apply(lambda x: x[-1])

In [ ]:
df = df.drop("Cpu", axis=1)

In [ ]:
df["CPU Frequency"] = df["CPU Frequency"].str[:-3]

In [ ]:
df["Ram"] = df["Ram"].str[:-2]

In [ ]:
df["Ram"] = df["Ram"].astype("int")
df["CPU Frequency"] = df["CPU Frequency"].astype("float")

In [ ]:
df["Screen Width"] = df["Screen Width"].astype("int")
df["Screen Height"] = df["Screen Height"].astype("int")

In [ ]:
df["Memory Amount"] = df.Memory.str.split(" ").apply(lambda x: x[0])
df["Memory Type"] = df.Memory.str.split(" ").apply(lambda x: x[1])

In [ ]:
def turn_memory_into_MB(value):
    if "GB" in value:
        return float(value[:value.find("GB")]) * 1000
    elif "TB" in value:
        return float(value[:value.find("TB")]) * 1000000

In [ ]:
df["Memory Amount"] = df["Memory Amount"].apply(turn_memory_into_MB)

In [ ]:
df = df.drop("Memory", axis=1)

In [ ]:
df["Weight"] = df["Weight"].str[:-2]

In [ ]:
df["Weight"] = df["Weight"].astype("float")

In [ ]:
df["GPU Brand"] = df.Gpu.str.split(" ").apply(lambda x: x[0])

In [ ]:
df = df.drop("Gpu", axis=1)

In [ ]:
df = df.join(pd.get_dummies(df.OpSys))
df = df.drop("OpSys", axis=1)

In [ ]:
cpu_categories = pd.get_dummies(df["CPU Brand"])
cpu_categories.columns = [col + "_CPU" for col in cpu_categories.columns]

df = df.join(cpu_categories)
df = df.drop("CPU Brand", axis=1)

In [ ]:
gpu_categories = pd.get_dummies(df["GPU Brand"])
gpu_categories.columns = [col + "_GPU" for col in gpu_categories.columns]

df = df.join(gpu_categories)
df = df.drop("GPU Brand", axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 15))
sns.heatmap(df.corr(), annot=True, cmap="YlGnBu")

In [ ]:
target_correlations = df.corr()['Price_euros'].apply(abs).sort_values()

In [ ]:
target_correlations

In [ ]:
selected_features = target_correlations[-21:].index

In [ ]:
selected_features = list(selected_features)

In [ ]:
selected_features

In [ ]:
limited_df = df[selected_features]

In [ ]:
limited_df

In [ ]:
plt.figure(figsize=(18, 15))
sns.heatmap(limited_df.corr(), annot=True, cmap="YlGnBu")

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

X, y = limited_df.drop("Price_euros", axis=1), limited_df["Price_euros"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
forest = RandomForestRegressor()

forest.fit(X_train_scaled, y_train)

In [ ]:
forest.score(X_test_scaled, y_test)

In [ ]:
y_pred = forest.predict(X_test_scaled)

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(y_pred, y_test)
plt.plot(range(0, 6000), range(0, 6000), c="red")


In [ ]:
X_new_scaled = scaler.transform([X_test.iloc[0]])

In [ ]:
forest.predict(X_new_scaled)

In [ ]:
y_test.iloc[0]